In [3]:
import torch
import sys
import os
import time
import pickle
import copy
import numpy as np
import pandas as pd
import torch.nn as nn
from torch import cuda
from torch.autograd import Variable
from gensim.models import KeyedVectors
from math import ceil
from hyperparams_glove import Hyperparams as hp
#from hyperparams_gn_glove import Hyperparams as hp
import random
import pre_train_autoencoder
import pre_train_classifier
import optim
import model
import matplotlib.pyplot as plt
%matplotlib inline
from torch.utils.data import TensorDataset, DataLoader

from livelossplot import PlotLosses
liveloss = PlotLosses()

In [4]:
import model

In [5]:
from pre_train_autoencoder import pre_train_autoencoder
from pre_train_classifier import pre_train_classifier

In [ ]:
from hyperparams_glove import Hyperparams as hp

In [6]:
print('loading word embedding')
#binary is false as data is in text format
emb = KeyedVectors.load_word2vec_format(hp.word_embedding, binary=hp.emb_binary)

loading word embedding


In [7]:
def remove_words_not_in_word2emb(emb,words):
    return [word for word in words if word in emb]
def make_no_gender_words(f,emb):
    return remove_words_not_in_word2emb(emb,[l.strip() for l in f])

def remove_pairs_not_in_word2emb(emb,pairs):
    return [word1 for word1,word2 in pairs if word1 in emb and word2 in emb],\
        [word2 for word1,word2 in pairs if word2 in emb and word1 in emb]
def make_G_pair_words(f,emb):
    df = pd.read_csv(f)
    female_words = df.iloc[:,0].values.tolist()
    male_words = df.iloc[:,1].values.tolist()
    
    if len(female_words)==len(male_words):
        return remove_pairs_not_in_word2emb(emb,[[word1,word2] for word1,word2 in zip(female_words,male_words)])
    elif len(female_words)!=len(male_words):
        return remove_words_not_in_word2emb(emb,female_words), remove_words_not_in_word2emb(emb, male_words)
def make_pair_words(f,emb):
    df = pd.read_csv(f, sep='\t')
    female_words = df.iloc[:,0].values.tolist()
    male_words = df.iloc[:,1].values.tolist()
    
    if len(female_words)==len(male_words):
        return remove_pairs_not_in_word2emb(emb,[[word1,word2] for word1,word2 in zip(female_words,male_words)])
    elif len(female_words)!=len(male_words):
        return remove_words_not_in_word2emb(emb,female_words), remove_words_not_in_word2emb(emb, male_words)

In [8]:
def shuffle_data(words, tags=None, limit_size=None, sampling=None):
    perm = torch.randperm(len(words))
    words = [words[idx.item()] for idx in perm]
    return words, tags

In [9]:
def shuffle_data1(words, p_pos):
    p_pos = list(p_pos)
    perm = np.random.choice(list(range(len(words))),size=len(words),p=p_pos)
    sorted_inds = np.sort(perm)
    words = [words[idx.item()] for idx in sorted_inds]
    return words

In [10]:
def create_train_dev(gender_words,no_gender_words,stereotype_words):
    stereotype_words=stereotype_words['female']+stereotype_words['male']
    gender_pairs = [[female,male] for female,male in zip(gender_words['female'], gender_words['male'])]
    
    no_gender_words,_ = shuffle_data(no_gender_words)
    gender_pairs,_ = shuffle_data(gender_pairs)
    stereotype_words,_ = shuffle_data(stereotype_words)
    train_words={}
    train_words['no_gender'] = no_gender_words[:-hp.dev_num]
    train_words['female&male'] = gender_pairs[:-hp.dev_num]
    train_words['stereotype'] = stereotype_words[:-hp.dev_num]
    
    dev_words={}
    dev_words['no_gender'] = no_gender_words[-hp.dev_num:]
    dev_words['female&male'] = gender_pairs[-hp.dev_num:]
    dev_words['stereotype'] = stereotype_words[-hp.dev_num:]
    return train_words,dev_words
    

In [11]:
def make_optim(model, optimizer, learning_rate, lr_decay, max_grad_norm):
    model_optim = optim.Optim(optimizer, learning_rate, lr_decay, max_grad_norm)
    model_optim.set_parameters(model.parameters())
    return model_optim


In [12]:
print("loading data")
stereotype_words={}
gender_words={}
#making sure that our no gender words are present in embedding
no_gender_words=make_no_gender_words(open(hp.no_gender_words), emb)
stereotype_words['female'],stereotype_words['male'] = make_pair_words(hp.stereotype_words,emb)
gender_words['female'],gender_words['male']=make_G_pair_words(hp.gender_words,emb )
all_words =no_gender_words+stereotype_words['female']+stereotype_words['male']+gender_words['female']+gender_words['male']
train_words,dev_words = create_train_dev(gender_words, no_gender_words,stereotype_words)

loading data


In [13]:
word2emb={}
for word in all_words:
    word2emb[word] = emb[word]

In [14]:
print(all_words)

['abandonment', 'abate', 'aberrant', 'abiding', 'able', 'abolition', 'abomination', 'abrupt', 'absorbing', 'absorption', 'abstention', 'abstraction', 'absurd', 'absurdity', 'abundance', 'abundantly', 'accept', 'acceptable', 'access', 'accident', 'accidentally', 'accompany', 'accomplish', 'according', 'accordingly', 'account', 'accumulation', 'accurate', 'accuse', 'achieve', 'achievement', 'acid', 'acknowledge', 'acquire', 'actuality', 'adaptable', 'adaptation', 'addictive', 'adherence', 'adjacent', 'adjustable', 'adjustment', 'adjustments', 'admittedly', 'ado', 'adorable', 'adore', 'adorn', 'advancement', 'advent', 'adverse', 'adversity', 'advertisement', 'aerial', 'afar', 'affected', 'afternoon', 'ago', 'agonies', 'agree', 'agreement', 'agricultural', 'air', 'aircraft', 'airliner', 'airport', 'alarms', 'alongside', 'aloof', 'alternately', 'amazing', 'amazingly', 'amount', 'amusement', 'analogous', 'analyses', 'answer', 'apartments', 'apparatus', 'apparent', 'apparently', 'appetizer', 

In [16]:
import random
import torch


torch.manual_seed(0)
random.seed(0)

def pre_train_autoencoder(hp,encoder, encoder_optim,decoder, decoder_optim, train_words, dev_words, word2emb,shuffle_data1):

    train_size  = len(train_words['no_gender']) + len(train_words['female&male'])+ len(train_words['stereotype'])
    dev_size = len(dev_words['no_gender']) + len(dev_words['female&male']) +len(dev_words['stereotype'])
    classifier_criterion = nn.BCELoss()
    decoder_criterion = nn.MSELoss()
    encoder1=encoder
    decode2 =decoder
    def vae_loss_function(mu, logsigma, kl_weight=0.0001):
  
        latent_loss = 0.5*torch.sum(logsigma.exp() + mu.pow(2)-1-logsigma)
        vae_loss = kl_weight*latent_loss 
        #print(vae_loss)
        return vae_loss
    def sampling(z_mean,z_logsigma):
    #print(hp.hid.shape)
    #print(z_logsigma)
        batch, latent_dim = z_mean.shape
        #epsilon = torch.distributions.Normal(torch.tensor([0.0]), torch.tensor([1.0]),sample_shape=(batch,latent_dim))
        epsilon = torch.rand((batch, latent_dim))

        z = z_mean + torch.exp(0.5*z_logsigma)*epsilon
        #print(z.shape)
        return z
    def encode(x):
        #encoder.zero_grad()
        
        for i,words in enumerate(x):
            emb1 =  torch.stack([torch.from_numpy(word2emb[word]) for word in words])
            encoder_output = encoder(emb1)
        z_mean = encoder_output[:,:hp.hidden_size//2]
       
        z_logsigma = encoder_output[:,hp.hidden_size//2:]
        
        return encoder_output, z_mean, z_logsigma
    def encoder_1(emb_1):
        encoder_output = encoder(emb_1)
        z_mean = encoder_output[:,:hp.hidden_size//2]
       
        z_logsigma = encoder_output[:,hp.hidden_size//2:]
        
        return encoder_output, z_mean, z_logsigma
    
    def get_latent_mu(words,batch_size=4):
        
        N = len(words)
        mu = np.zeros((N, hp.hidden_size//2))
        for start_ind in range(0, N, batch_size):
            end_ind = min(start_ind+batch_size, N+1)
            if end_ind%batch_size!=0:
                break
            batch = (words[start_ind:end_ind])
            _,batch_mu, _ = encode(batch)
            
            mu[start_ind:end_ind] = batch_mu.detach().numpy()
        return mu
    def get_training_sample_probabilities(train_words,bins=4, smoothing_fac=0.005):
        mu = get_latent_mu(train_words)
        training_sample_p = np.zeros(mu.shape[0])
        for i in range(hp.hidden_size//2):
            latent_distribution = mu[:,i]
            # generate a histogram of the latent distribution
            hist_density, bin_edges =  np.histogram(latent_distribution, density=True, bins=bins)

            # find which latent bin every data sample falls in 
            bin_edges[0] = -float('inf')
            bin_edges[-1] = float('inf')

            # TODO: call the digitize function to find which bins in the latent distribution 
            #    every data sample falls in to
            # https://docs.scipy.org/doc/numpy-1.13.0/reference/generated/numpy.digitize.html
            bin_idx = np.digitize(latent_distribution, bin_edges) # TODO

            # smooth the density function
            hist_smoothed_density = hist_density + smoothing_fac
            hist_smoothed_density = hist_smoothed_density / np.sum(hist_smoothed_density)

            # invert the density function 
            p = 1.0/(hist_smoothed_density[bin_idx-1])

            # TODO: normalize all probabilities
            p = p/np.sum(p)

            # TODO: update sampling probabilities by considering whether the newly
            #     computed p is greater than the existing sampling probabilities.
            training_sample_p = np.maximum(p, training_sample_p)
        
    # final normalization
        training_sample_p /= np.sum(training_sample_p)

        return training_sample_p
    def run_model(words, mode,p_in):
        #batch_size=4
        if mode=="train":
            encoder.train()
            decoder.train()
        elif model == 'eval':
            encoder.eval()
            decoder.eval()
        total_loss = 0
        total_num = 0
        total_decoder_loss = 0
        total_vae_loss = 0
        vae_loss=0
        if mode=='train':
            limit_size =  max([len(words['no_gender']), len(words['female&male']),
                               len(words['stereotype'])])
            words['no gender'],_ = \
                shuffle_data(words['no_gender'],
                             limit_size=limit_size,
                             sampling = hp.sampling)
            words['female & male'],_ = \
                shuffle_data(words['female&male'],
                             limit_size=limit_size,
                             sampling=hp.sampling)
            words['stereotype'],_ = \
                shuffle_data(words['stereotype'],
                             limit_size=limit_size,
                             sampling=hp.sampling)
        elif mode=='eval':
            limit_size = hp.dev_num
        
        inputs = [[gender[0], gender[1], stereotype, no_gender] for gender, stereotype, no_gender in zip(
            words['female&male'], words['stereotype'], words['no_gender'])]
        if mode=='eval':
            x=inputs
        else:
            x = shuffle_data1(inputs,p_in)
        
        data_size = len(x)*4
        b_s = 4
        for j in range(len(inputs)//4):
            if mode=="train":
                y=x[:32]
            if mode=='eval':
                #j=4
                y=inputs
            for i,words in enumerate(y):
                #print(y)
                #z_mean,z_logsigma,recon = call(batch)
                #vae_loss_1  = debiasing_loss_function(z_mean,z_logsigma)
                emb_dummy =  torch.stack([torch.from_numpy(word2emb[word]) for word in words])
                encoder.zero_grad()
                hidden, z_mean,z_logsigma = encoder_1(emb_dummy)
                z = sampling(z_mean,z_logsigma)
                decoder.zero_grad()
                pre = decoder(z)
                vae_loss = vae_loss_function(z_mean,z_logsigma)
                decoder_loss = decoder_criterion(pre, emb_dummy)
                #vae_loss *= (0.0005)
                loss = decoder_loss+ vae_loss
                #print(loss)
                #total_vae_loss+=vae_loss_1.item()
                if mode=='train':
                    loss.backward()
                    encoder_optim.step()
                    decoder_optim.step()
            total_loss += loss.item()
            #total_num += len(y)
        return total_loss/data_size
                    

    
    print('start training')
    best_loss = float('inf')
    decoder_loss_list=[]
    female_classifier_loss_list=[]
    male_classifier_loss_list=[]
    gender_stereotype_loss_list = []
    gender_no_gender_loss_list = []
    gender_vektor_loss_list = []
    total_loss_list = []
    female_classifier_acc_list = []
    male_classifier_acc_list = []
    vae_loss_list=[]
    logs={}
    inputs1 = [[gender[0], gender[1], stereotype, no_gender] for gender, stereotype, no_gender in zip(train_words['female&male'], train_words['stereotype'], train_words['no_gender'])]
    p_inp = get_training_sample_probabilities(inputs1)
        #l1=[]
    min_loss = float('inf')
    for epoch in range(300):
        train_loss = run_model(train_words, mode='train',p_in=p_inp)
        eval_loss = run_model(dev_words, mode='eval', p_in=p_inp)
        print(epoch)
    if eval_loss < min_loss:
        min_epoch = epoch
        min_loss = eval_loss
        encoder_state_dict = encoder.state_dict()
        decoder_state_dict = decoder.state_dict()
        checkpoint = {
            'encoder': encoder_state_dict,
            'decoder': decoder_state_dict,
            'hp': hp
        }
        torch.save(checkpoint,
            '{}autoencoder_checkpoint'.format('./'))

    checkpoint = torch.load('{}autoencoder_checkpoint'.format('./'))
    torch.save(checkpoint, '{}autoencoder.pt'.format('./'))

    import os
    os.remove('{}autoencoder_checkpoint'.format('./'))

    return checkpoint

True

In [18]:
if hp.pre_train_autoencoder:
        print('Pre-training autoencoder')
        encoder = model.Encoder(hp.emb_size, hp.hidden_size, hp.pta_dropout_rate)
        decoder = model.Decoder(150, hp.emb_size, hp.pta_dropout_rate)
        print(decoder)
        encoder_optim = make_optim(encoder,
                                     hp.pta_optimizer,
                                     hp.pta_learning_rate,
                                     hp.pta_lr_decay,
                                     hp.pta_max_grad_norm)
        decoder_optim = make_optim(decoder,
                                            hp.pta_optimizer,
                                            hp.pta_learning_rate,
                                            hp.pta_lr_decay,
                                            hp.pta_max_grad_norm)
        if hp.pre_data == 'random':
            checkpoint = pre_train_autoencoder(hp,encoder, encoder_optim,decoder, decoder_optim, train_words, dev_words, word2emb,shuffle_data1)
        elif hp.pre_data == 'common':
            checkpoint = pre_train_autoencoder(hp,
                                          encoder,
                                          encoder_optim,
                                          decoder,
                                          decoder_optim,
                                          emb,
                                          dev_words=dev_words)

Pre-training autoencoder
Decoder(
  (output_layer): Linear(in_features=150, out_features=300, bias=True)
  (output_layer1): Linear(in_features=300, out_features=300, bias=True)
  (tanh): Tanh()
)
start training
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224

In [19]:
import random
import torch


torch.manual_seed(0)
random.seed(0)

def save_checkpoint(hp, mode, model):
    model_state_dict = model.state_dict()
    checkpoint = {
        mode: model_state_dict,
        'hp': hp
    }
    torch.save(checkpoint,
        '{}{}_checkpoint'.format('./', mode))

def pre_train_classifier(hp,
                         female_classifier,
                         male_classifier,
                         female_optim,
                         male_optim,
                         train_female_embs,
                         train_male_embs,
                         train_stereotype_embs,
                         dev_female_embs,
                         dev_male_embs,
                         dev_stereotype_embs):

    criterion = torch.nn.MSELoss()

    def run_model(female_embs, male_embs, no_gender_embs, mode):
        inputs = female_embs + male_embs + no_gender_embs
        tags = [0 for _ in range(len(female_embs))] + [1 for _ in range(len(male_embs))] + [2 for _ in range(len(no_gender_embs))]

        if mode == 'train':
            female_classifier.train()
            male_classifier.train()
            perm = [idx for idx in range(len(inputs))]
            random.shuffle(perm)
            inputs = [inputs[idx] for idx in perm]
            tags = [tags[idx] for idx in perm]
        elif mode == 'eval':
            female_classifier.eval()
            male_classifier.eval()
        total_female_num = 0
        total_male_num = 0
        total_female_loss = 0
        total_male_loss = 0
        for input, tag in zip(inputs, tags):
            zero_tag = torch.zeros(1)
            one_tag = torch.ones(1)
            female_classifier.zero_grad()
            male_classifier.zero_grad()
            female_pre = female_classifier(input)
            male_pre = male_classifier(input)
            if tag == 0:
                female_loss = criterion(female_pre, one_tag)
                male_loss = criterion(male_pre, zero_tag)
            elif tag == 1:
                female_loss = criterion(female_pre, zero_tag)
                male_loss = criterion(male_pre, one_tag)
            elif tag == 2:
                female_loss = criterion(female_pre, zero_tag)
                male_loss = criterion(male_pre, zero_tag)
                #continue
            if mode == 'train':
                female_loss.backward()
                male_loss.backward()
                female_optim.step()
                male_optim.step()
            total_female_loss += female_loss.item()
            total_male_loss += male_loss.item()
            total_female_num += len(female_embs)
            total_male_num += len(male_embs)

        return total_female_loss / total_female_num, total_male_loss / total_male_num


    min_female_loss = float('inf')
    min_male_loss = float('inf')
    for epoch in range(1, 100):
        train_female_loss, train_male_loss = run_model(train_female_embs, train_male_embs, train_stereotype_embs, 'train')
        eval_female_loss, eval_male_loss = run_model(dev_female_embs, dev_male_embs, dev_stereotype_embs, 'eval')

        if eval_female_loss < min_female_loss:
            min_female_epoch = epoch
            min_female_loss = eval_female_loss
            female_state_dict = female_classifier.state_dict()
            female_checkpoint = {
                'female': female_state_dict,
                'hp': hp
            }
            torch.save(female_checkpoint,
                '{}female_checkpoint'.format('./'))
        if eval_male_loss < min_male_loss:
            min_male_epoch = epoch
            min_male_loss = eval_male_loss
            male_state_dict = male_classifier.state_dict()
            male_checkpoint = {
                'male': male_state_dict,
                'hp': hp
            }
            torch.save(male_checkpoint,
                '{}male_checkpoint'.format('./'))

    female_checkpoint = torch.load('{}female_checkpoint'.format('./'))
    torch.save(female_checkpoint, '{}female.pt'.format('./'))
    male_checkpoint = torch.load('{}male_checkpoint'.format('./'))
    torch.save(male_checkpoint, '{}male.pt'.format('./'))

    import os
    os.remove('{}female_checkpoint'.format('./'))
    os.remove('{}male_checkpoint'.format('./'))

    return female_checkpoint, male_checkpoint

In [20]:
encoder=model.Encoder(hp.emb_size, hp.hidden_size, hp.dropout_rate)
decoder =model.Decoder(hp.hidden_size//2, hp.emb_size, hp.dropout_rate)

if hp.pre_train_autoencoder:
    encoder.load_state_dict(checkpoint['encoder'])
    decoder.load_state_dict(checkpoint['decoder'])

In [21]:


if hp.pre_train_classifier:
    female_classifier = model.Classifier(hp.hidden_size)
    male_classifier = model.Classifier(hp.hidden_size)
    female_classifier_optim = make_optim(female_classifier,
                                 hp.cls_optimizer,
                                 hp.cls_learning_rate,
                                 hp.cls_lr_decay,
                                 hp.cls_max_grad_norm)
    male_classifier_optim = make_optim(male_classifier,
                                        hp.cls_optimizer,
                                        hp.cls_learning_rate,
                                        hp.cls_lr_decay,
                                        hp.cls_max_grad_norm)

    encoder.eval()
    encoder.zero_grad()

    train_females = []
    train_males = []
    dev_females = []
    dev_males = []

    train_female_embs = [encoder(torch.FloatTensor(emb[word[0]])).data for word in train_words['female&male']]
    encoder.zero_grad()
    train_male_embs = [encoder(torch.FloatTensor(emb[word[1]])).data for word in train_words['female&male']]
    encoder.zero_grad()
    train_stereotype_embs = [encoder(torch.FloatTensor(emb[word])).data for word in train_words['no_gender']]
    encoder.zero_grad()

    dev_female_embs = [encoder(torch.FloatTensor(emb[word[0]])).data for word in dev_words['female&male']]
    encoder.zero_grad()
    dev_male_embs = [encoder(torch.FloatTensor(emb[word[1]])).data for word in dev_words['female&male']]
    encoder.zero_grad()
    dev_stereotype_embs = [encoder(torch.FloatTensor(emb[word])).data for word in dev_words['no_gender']]
    encoder.zero_grad()

    print('Pre-training classifier')
    female_checkpoint, male_checkpoint = pre_train_classifier(hp,
                                        female_classifier,
                                        male_classifier,
                                        female_classifier_optim,
                                        male_classifier_optim,
                                        train_female_embs,
                                        train_male_embs,
                                        train_stereotype_embs,
                                        dev_female_embs,
                                        dev_male_embs,
                                        dev_stereotype_embs)

Pre-training classifier


In [22]:
print('Building female & male classifiers')
female_classifier = model.Classifier(hp.hidden_size)
male_classifier = model.Classifier(hp.hidden_size)

if hp.pre_train_classifier:
    female_classifier.load_state_dict(female_checkpoint['female'])
    male_classifier.load_state_dict(male_checkpoint['male'])

print('Setting optimizer')

Building female & male classifiers
Setting optimizer


In [23]:
encoder=model.Encoder(hp.emb_size, hp.hidden_size, hp.dropout_rate)
decoder =model.Decoder(hp.hidden_size//2, hp.emb_size, hp.dropout_rate)


print('Building female & male classifiers')
female_classifier = model.Classifier(hp.hidden_size)
male_classifier = model.Classifier(hp.hidden_size)


print("setting optimizer")
encoder_optim = make_optim(encoder,
                                 hp.optimizer,
                                 hp.learning_rate,
                                 hp.lr_decay,
                                 hp.max_grad_norm)
female_classifier_optim = make_optim(female_classifier,
                                    hp.optimizer,
                                    hp.learning_rate,
                                    hp.lr_decay,
                                    hp.max_grad_norm)
male_classifier_optim = make_optim(male_classifier,
                                    hp.optimizer,
                                    hp.learning_rate,
                                    hp.lr_decay,
                                    hp.max_grad_norm)
decoder_optim = make_optim(decoder,
                                    hp.optimizer,
                                    hp.learning_rate,
                                    hp.lr_decay,
                                    hp.max_grad_norm)

Building female & male classifiers
setting optimizer


In [24]:
def vae_loss_function(mu, logsigma, kl_weight=0.0001):
  
    latent_loss = 0.5*torch.sum(logsigma.exp() + mu.pow(2)-1-logsigma)
    vae_loss = kl_weight*latent_loss 
    #print(vae_loss)
    return vae_loss

In [25]:
def debiasing_loss_function(mu, logsigma):

    vae_loss = vae_loss_function(mu,logsigma) # TODO

    total_loss = tf.reduce_mean(vae_loss)

    return total_loss

In [26]:
def sampling(z_mean,z_logsigma):
    #print(hp.hid.shape)
    #print(z_logsigma)
    batch, latent_dim = z_mean.shape
    epsilon = torch.rand((batch, latent_dim))
    
    z = z_mean + torch.exp(0.5*z_logsigma)*epsilon
    #print(z.shape)
    return z

In [28]:
train(encoder, encoder_optim, female_classifier, female_classifier_optim,male_classifier, male_classifier_optim,
           decoder, decoder_optim, train_words, dev_words, word2emb,shuffle_data1,sampling)

start training


In [27]:
def train(encoder, encoder_optim, female_classifier, female_classifier_optim,male_classifier, male_classifier_optim,
           decoder, decoder_optim, train_words, dev_words, word2emb,shuffle_data1,sampling):
    
    train_size  = len(train_words['no_gender']) + len(train_words['female&male'])+ len(train_words['stereotype'])
    dev_size = len(dev_words['no_gender']) + len(dev_words['female&male']) +len(dev_words['stereotype'])
    classifier_criterion = nn.BCELoss()
    decoder_criterion = nn.MSELoss()
    encoder1=encoder
    decode2 =decoder
    def encode(x):
        #encoder.zero_grad()
        
        for i,words in enumerate(x):
            emb1 =  torch.stack([torch.from_numpy(word2emb[word]) for word in words])
            encoder_output = encoder(emb1)
        z_mean = encoder_output[:,:hp.hidden_size//2]
       
        z_logsigma = encoder_output[:,hp.hidden_size//2:]
        
        return encoder_output, z_mean, z_logsigma
    def encoder_1(emb_1):
        encoder_output = encoder(emb_1)
        z_mean = encoder_output[:,:hp.hidden_size//2]
       
        z_logsigma = encoder_output[:,hp.hidden_size//2:]
        
        return encoder_output, z_mean, z_logsigma
    
    def get_latent_mu(words,batch_size=4):
        
        N = len(words)
        mu = np.zeros((N, hp.hidden_size//2))
        for start_ind in range(0, N, batch_size):
            end_ind = min(start_ind+batch_size, N+1)
            if end_ind%batch_size!=0:
                break
            batch = (words[start_ind:end_ind])
            _,batch_mu, _ = encode(batch)
            
            mu[start_ind:end_ind] = batch_mu.detach().numpy()
        return mu
    def get_training_sample_probabilities(train_words,bins=4, smoothing_fac=0.005):
        mu = get_latent_mu(train_words)
        training_sample_p = np.zeros(mu.shape[0])
        for i in range(hp.hidden_size//2):
            latent_distribution = mu[:,i]
            # generate a histogram of the latent distribution
            hist_density, bin_edges =  np.histogram(latent_distribution, density=True, bins=bins)

            # find which latent bin every data sample falls in 
            bin_edges[0] = -float('inf')
            bin_edges[-1] = float('inf')

            #call the digitize function to find which bins in the latent distribution every data sample falls in to
            # https://docs.scipy.org/doc/numpy-1.13.0/reference/generated/numpy.digitize.html
            bin_idx = np.digitize(latent_distribution, bin_edges) 

            # smooth the density function
            hist_smoothed_density = hist_density + smoothing_fac
            hist_smoothed_density = hist_smoothed_density / np.sum(hist_smoothed_density)

            # invert the density function 
            p = 1.0/(hist_smoothed_density[bin_idx-1])

            # TODO: normalize all probabilities
            p = p/np.sum(p)

            # update sampling probabilities by considering whether the newly
            #     computed p is greater than the existing sampling probabilities.
            training_sample_p = np.maximum(p, training_sample_p)
        
    # final normalization
        training_sample_p /= np.sum(training_sample_p)

        return training_sample_p
    def run_model(words, gender_vektor, mode,p_in):
        #batch_size=4
        if mode=="train":
            encoder.train()
            female_classifier.train()
            male_classifier.train()
            decoder.train()
        elif model == 'eval':
            encoder.eval()
            female_classifier.eval()
            male_classifier.eval()
            decoder.eval()
        total_loss = 0
        total_female_classifier_loss = 0
        total_male_classifier_loss = 0
        total_decoder_loss = 0
        total_gender_stereotype_loss = 0
        total_gender_no_gender_loss = 0
        total_gender_vektor_loss = 0
        total_vae_loss = 0
        female_classifier_correct = 0
        male_classifier_correct = 0
        vae_loss=0
        if mode=='train':
            limit_size =  max([len(words['no_gender']), len(words['female&male']),
                               len(words['stereotype'])])
            words['no gender'],_ = \
                shuffle_data(words['no_gender'],
                             limit_size=limit_size,
                             sampling = hp.sampling)
            words['female & male'],_ = \
                shuffle_data(words['female&male'],
                             limit_size=limit_size,
                             sampling=hp.sampling)
            words['stereotype'],_ = \
                shuffle_data(words['stereotype'],
                             limit_size=limit_size,
                             sampling=hp.sampling)
        elif mode=='eval':
            limit_size = hp.dev_num
        
        inputs = [[gender[0], gender[1], stereotype, no_gender] for gender, stereotype, no_gender in zip(
            words['female&male'], words['stereotype'], words['no_gender'])]
        if mode=='eval':
            x=inputs
        else:
            x = shuffle_data1(inputs,p_in)
        
        data_size = len(x)*4
        def classify(hidden, gold, classifier):
            classifier.zero_grad()
            pre = classifier(hidden)
            loss = classifier_criterion(pre,gold)
            return pre, loss
        b_s = 4
        for j in range(len(inputs)//4):
            if mode=="train":
                y=x[:32]
            if mode=='eval':
                #j=4
                y=inputs
            for i,words in enumerate(y):
                #print(y)
                #z_mean,z_logsigma,recon = call(batch)
                #vae_loss_1  = debiasing_loss_function(z_mean,z_logsigma)
                emb_dummy =  torch.stack([torch.from_numpy(word2emb[word]) for word in words])
                encoder.zero_grad()
                hidden, z_mean,z_logsigma = encoder_1(emb_dummy)
                z = sampling(z_mean,z_logsigma)
                decoder.zero_grad()
                pre = decoder(z)
                vae_loss = vae_loss_function(z_mean,z_logsigma)
                decoder_loss = decoder_criterion(pre, emb_dummy)
                
                female_gold = torch.FloatTensor([1,0,0,0]).view(-1,1)
                male_gold = torch.FloatTensor([0,1,0,0]).view(-1,1)
                female_pre, female_loss = classify(hidden, female_gold, female_classifier)
                
                male_pre, male_loss = classify(hidden, male_gold, male_classifier)
                gender_stereotype_loss = torch.sum(gender_vektor*hidden[2])**2
                gender_no_gender_loss = torch.sum(gender_vektor*hidden[3])**2
                gender_vektor_loss  = decoder_criterion(gender_vektor,(hidden[1]-hidden[0]))
                decoder_loss *=hp.decoder_loss_rate
                female_loss*=hp.female_loss_rate
                male_loss *=hp.male_loss_rate
                gender_stereotype_loss*=hp.gender_stereotype_loss_rate
                gender_no_gender_loss*=hp.gender_no_gender_loss_rate
                gender_vektor_loss *=hp.gender_vektor_loss_rate
                #vae_loss *= (0.0005)
                loss = decoder_loss+female_loss+male_loss+gender_stereotype_loss+gender_no_gender_loss+gender_vektor_loss+vae_loss
                #print(loss)
                total_decoder_loss+=decoder_loss.item()
                total_gender_stereotype_loss+=gender_stereotype_loss.item()
                total_female_classifier_loss += female_loss.item()
                total_male_classifier_loss += male_loss.item()
                total_gender_no_gender_loss+=gender_no_gender_loss.item()
                total_gender_vektor_loss+=gender_vektor_loss.item()
                total_vae_loss+=vae_loss.item()
                #total_vae_loss+=vae_loss_1.item()
                total_loss+=loss.item()
            
                female_classifier_correct +=torch.sum(torch.eq(female_gold,(female_pre>0.5).float().view(-1))).item()
                male_classifier_correct+=torch.sum(torch.eq(male_gold,(male_pre>0.5).float().view(-1))).item()
                if mode=='train':
                    loss.backward()
                    encoder_optim.step()
                    female_classifier_optim.step()
                    male_classifier_optim.step()
                    decoder_optim.step()
                
        return total_decoder_loss/data_size, total_female_classifier_loss/data_size,total_male_classifier_loss/data_size,total_gender_stereotype_loss / data_size, total_gender_no_gender_loss / data_size,total_gender_vektor_loss / data_size,total_loss / data_size,female_classifier_correct / data_size, male_classifier_correct / data_size,total_vae_loss/data_size

    def calculate_gender_vektor(encoder, word2emb, gender_pairs):
        #encoder.eval()
        #encoder.zero_grad()
        females=[]
        males=[]
        for female, male in gender_pairs:
            females+=[female]
            males+=[male]
        #print(gender_pairs)
        female_embs = torch.stack([encoder(torch.FloatTensor(emb[word])).data for word in females], dim=0)
        male_embs = torch.stack([encoder(torch.FloatTensor(emb[word])).data for word in males], dim=0)
        gender_vektor = torch.sum(male_embs-female_embs,0)/male_embs.size(0)
        return gender_vektor
    
    print('start training')
    best_loss = float('inf')
    decoder_loss_list=[]
    female_classifier_loss_list=[]
    male_classifier_loss_list=[]
    gender_stereotype_loss_list = []
    gender_no_gender_loss_list = []
    gender_vektor_loss_list = []
    total_loss_list = []
    female_classifier_acc_list = []
    male_classifier_acc_list = []
    vae_loss_list=[]
    for epochs in range(5):
        logs={}
        inputs1 = [[gender[0], gender[1], stereotype, no_gender] for gender, stereotype, no_gender in zip(
            train_words['female&male'], train_words['stereotype'], train_words['no_gender'])]
        p_inp = get_training_sample_probabilities(inputs1)
        gender_vektor = calculate_gender_vektor(encoder,emb, train_words['female&male'])
        #l1=[]
        for j in range(25):
            results = run_model(copy.deepcopy(train_words), gender_vektor, mode='train', p_in=p_inp)
            decoder_loss_list += [results[0]]
            female_classifier_loss_list += [results[1]]
            male_classifier_loss_list += [results[2]]
            gender_stereotype_loss_list += [results[3]]
            if hp.gender_no_gender_loss:
                gender_no_gender_loss_list += [results[4]]
            if hp.gender_vektor_loss:
                gender_vektor_loss_list += [results[5]]
            total_loss_list += [results[6]]
            
            female_classifier_acc_list += [results[7]]
            male_classifier_acc_list += [results[8]]
            vae_loss_list+=[results[9]]
            
            #liveloss.update(logs)
            #liveloss.send()
        results = run_model(copy.deepcopy(dev_words), gender_vektor, mode='eval',p_in=p_inp)
        
        decoder_loss_list += [results[0]]
        female_classifier_loss_list += [results[1]]
        male_classifier_loss_list += [results[2]]
        gender_stereotype_loss_list += [results[3]]
        if hp.gender_no_gender_loss:
            gender_no_gender_loss_list += [results[4]]
        if hp.gender_vektor_loss:
            gender_vektor_loss_list += [results[5]]
        total_loss = results[6]
        #print("eval_loss",total_loss)
        total_loss_list += [total_loss]
        female_classifier_acc_list += [results[7]]
        male_classifier_acc_list += [results[8]]
        vae_loss_list+=[results[9]]
        if total_loss < best_loss:
            best_epoch = epochs
            best_loss = total_loss
            encoder_state_dict = encoder.state_dict()
            checkpoint = {
                'encoder': encoder_state_dict,
                'hp': hp,
                'encoder_optim': encoder_optim
            }
            torch.save(checkpoint, '{}model_checkpoint'.format('./'))
    checkpoint = torch.load('{}model_checkpoint'.format('./'))
    torch.save(checkpoint,
               '{}best_model.pt'.format('./', best_loss, best_epoch))
    #loss_history.append(total_loss_list.numpy().mean())